<a href="https://colab.research.google.com/github/kame-code-ai/llm/blob/main/embedding_models_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers FlagEmbedding tensorflow-text tensorflow_hub

In [ ]:
from sentence_transformers import SentenceTransformer, util
from FlagEmbedding import BGEM3FlagModel
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np

# Load the pre-trained models
sarashina_model = SentenceTransformer("sbintuitions/sarashina-embedding-v1-1b")
bge_m3_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
use_model_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3'
use_model = hub.KerasLayer(use_model_url, trainable=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
print(sarashina_model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: LlamaModel 
  (1): Pooling({'word_embedding_dimension': 1792, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': True, 'include_prompt': False})
)


In [ ]:
# Define the sentence pairs and expected similarity scores
sentence_pairs = [
    (["富士山は日本で一番高い山です。", "日本では、富士山が一番高い山です。"], 5),  # High similarity
    (["私は猫が好きです。", "彼は犬を飼っています。"], 2),  # Moderate similarity
    (["コーヒーを飲みます。", "本を読みます。"], 0),  # Low similarity
    (["富士山は日本で一番高い山です。", "今日はカレーを食べました。"], 0),  # Very low similarity
    (["新幹線は速くて便利です。", "彼はピアノを弾くのが上手です。"], 0),  # Very low similarity
    (["明日は雨が降るでしょう。", "私は新しいパソコンを買いました。"], 0),  # Very low similarity
]

# Function to calculate accuracy for a given model
def calculate_accuracy(model):
    total_pairs = len(sentence_pairs)
    correct_predictions = 0

    for pair, expected_score in sentence_pairs:
        # Encode the sentences
        if model == use_model:
            embeddings = model(pair).numpy()
        elif model == bge_m3_model:
            embeddings = model.encode(pair, batch_size=12, max_length=512,)['dense_vecs']
        else:
            embeddings = model.encode(pair)

        # Calculate similarity
        similarity_score = util.cos_sim(embeddings[0], embeddings[1]).item()

        # Round the similarity score to the nearest integer for comparison
        predicted_score = round(similarity_score * 5)  # Assuming a 0-5 similarity scale

        # Check if the prediction is correct
        if predicted_score == expected_score:
            correct_predictions += 1

        print(f"Sentence Pair: {pair}")
        print(f"Predicted Similarity: {predicted_score}")
        print(f"Expected Similarity: {expected_score}")
        print("-" * 20)

    # Calculate accuracy
    accuracy = (correct_predictions / total_pairs) * 100
    return accuracy

# Calculate accuracy for each model
sarashina_accuracy = calculate_accuracy(sarashina_model)
bge_m3_accuracy = calculate_accuracy(bge_m3_model)
use_accuracy = calculate_accuracy(use_model)

# Print the results
print(f"Sarashina Accuracy: {sarashina_accuracy:.2f}%")
print(f"BGE M3 Accuracy: {bge_m3_accuracy:.2f}%")
print(f"USE Accuracy: {use_accuracy:.2f}%")

Sentence Pair: ['富士山は日本で一番高い山です。', '日本では、富士山が一番高い山です。']
Predicted Similarity: 5
Expected Similarity: 5
--------------------
Sentence Pair: ['私は猫が好きです。', '彼は犬を飼っています。']
Predicted Similarity: 4
Expected Similarity: 2
--------------------
Sentence Pair: ['コーヒーを飲みます。', '本を読みます。']
Predicted Similarity: 4
Expected Similarity: 0
--------------------
Sentence Pair: ['富士山は日本で一番高い山です。', '今日はカレーを食べました。']
Predicted Similarity: 4
Expected Similarity: 0
--------------------
Sentence Pair: ['新幹線は速くて便利です。', '彼はピアノを弾くのが上手です。']
Predicted Similarity: 4
Expected Similarity: 0
--------------------
Sentence Pair: ['明日は雨が降るでしょう。', '私は新しいパソコンを買いました。']
Predicted Similarity: 4
Expected Similarity: 0
--------------------
Sentence Pair: ['富士山は日本で一番高い山です。', '日本では、富士山が一番高い山です。']
Predicted Similarity: 5
Expected Similarity: 5
--------------------
Sentence Pair: ['私は猫が好きです。', '彼は犬を飼っています。']
Predicted Similarity: 3
Expected Similarity: 2
--------------------
Sentence Pair: ['コーヒーを飲みます。', '本を読みます。']
Predicted Similarit